# This notebook can be executed after getRegionalBox.ipynb

In [1]:
using Pkg
using Metal

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv"  # maybe GeoPoints and planet1D should be fusioned

# batchGPU should be at this level (I have not made it as a module yet, since the choice of Metal/CUDA should be done in a manual way)
include("../src/batchFiles/batchGPU.jl")


include("../src/commonBatchs.jl")
include("../src/flexOPT.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

using .commonBatchs, .flexOPT, .planet1D, .GeoPoints

devs = Metal.devices() = Metal.MTL.MTLDeviceInstance[Metal.MTL.MTLDeviceInstance (object of type AGXG13XDevice)]

  Activating project at `~/Documents/Github/flexOPT`



→ Using Metal backend (1 device(s))
Selected backend type: MetalBackend


# give me a model of material variables (like seismic models)

In [2]:
# what you need as a model:

boxGrids = lazyProduceOrLoad("MarmousiCoordInfo"); # for a big model, I do not recommend to store these data

seismicModelMarmousi=lazyProduceOrLoad("seismicModelMarmousi");

This is the laziest ProduceOrLoad
Loading from ./tmp/MarmousiCoordInfo.jld2
This is the laziest ProduceOrLoad
Loading from ./tmp/seismicModelMarmousi.jld2


# choose your physics and get the semi symbolic OPT expression(s) 

In [3]:
famousEquationType="2DacousticTime"
@show myEquationInside=famousEquations(famousEquationType)


myEquationInside = famousEquations(famousEquationType) = (exprs = Differential(t)(Differential(t)(u(x, y, t))) - (Differential(x)(Differential(x)(u(x, y, t))) + Differential(y)(Differential(y)(u(x, y, t))))*(v(x, y)^2), fields = u(x, y, t), vars = v(x, y), extexprs = f(x, y, t), extfields = f(x, y, t), extvars = 1, coordinates = (x, y, t), ∂ = Any[Differential(x), Differential(y), Differential(t)], ∂² = Any[Differential(x) ∘ Differential(x), Differential(y) ∘ Differential(y), Differential(t) ∘ Differential(t)])


(exprs = Differential(t)(Differential(t)(u(x, y, t))) - (Differential(x)(Differential(x)(u(x, y, t))) + Differential(y)(Differential(y)(u(x, y, t))))*(v(x, y)^2), fields = u(x, y, t), vars = v(x, y), extexprs = f(x, y, t), extfields = f(x, y, t), extvars = 1, coordinates = (x, y, t), ∂ = Any[Differential(x), Differential(y), Differential(t)], ∂² = Any[Differential(x) ∘ Differential(x), Differential(y) ∘ Differential(y), Differential(t) ∘ Differential(t)])

In [4]:
Δnum = (1.0,1.0,1.0) # this should be in the same order as myEquationInside.coordinates 

# or
 
Δnum = (boxGrids.Δx,boxGrids.Δz,1.0) # hahaha this Δnum is not very good since there is no reason for the grids to be rectangular with this isotropic medium but let's try


(11.485642946317103, 4.444444444444445, 1.0)

# hereafter are the parameters which are important for modellers but maybe not interesting (so default numbers are given)

In [5]:
# concerning the order of test B-spline functions (-1 denotes a pulse function)
orderBtime=-1
orderBspace=-1
# the number of neighbouring points used to construct operators (except the point itself)
pointsInSpace=2
pointsInTime=2
# in order to control higher order partials, we need the supplemantary order > 0, for an ill-posed inversion of Taylor coefficients
supplementaryOrder=2
# B-spline order for the interpolated Taylor expansion
WorderBspace=-1
WorderBtime=-1

TaylorOptions=(WorderBtime=WorderBtime,WorderBspace=WorderBspace,supplementaryOrder=supplementaryOrder)
trialFunctionsCharacteristics=(orderBtime=orderBtime,orderBspace=orderBspace,pointsInSpace=pointsInSpace,pointsInTime=pointsInTime)



(orderBtime = -1, orderBspace = -1, pointsInSpace = 2, pointsInTime = 2)

In [6]:
OPTcoefConfig = @strdict myEquationInside=myEquationInside Δnum=Δnum TaylorOptions=TaylorOptions trialFunctionsCharacteristics=trialFunctionsCharacteristics 
coefs = myProduceOrLoad(OPTobj,OPTcoefConfig,"OPTcoefs") #attention! without GPU, OPTobj will take a lot of time (for >2D)

┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/OPTcoefs/OPTcoefs_5dbb41ef.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:106


(bigα, varM) = (Any[Any[(node = -(v(x, y)^2), nᶜ = CartesianIndex(1, 1, 1), n = CartesianIndex(3, 1, 1)), (node = -(v(x, y)^2), nᶜ = CartesianIndex(1, 1, 1), n = CartesianIndex(1, 3, 1)), (node = 1, nᶜ = CartesianIndex(1, 1, 1), n = CartesianIndex(1, 1, 3))];;], Any[v₁ v₂ v₃ v₄ v₅ v₆ v₇ v₈ v₉ v₁ v₂ v₃ v₄ v₅ v₆ v₇ v₈ v₉ v₁ v₂ v₃ v₄ v₅ v₆ v₇ v₈ v₉])


┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/intKernel/intKernel_aef046f3.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:106
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_d7523fb2.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:106
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/Bspline_4c5b706d.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:106
┌ Warning: The Git repository ('/Users/nobuaki/Documents/Github/flexOPT') is dirty! Appending -dirty to the commit ID.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_tools.jl:71
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/Bspline_4c5b706d.jld2 saved.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2

(μᶜ, μ, l_n_variable + 1, l_n_field + 1, kernels[μᶜ, μ, l_n_variable + 1, l_n_field + 1]) = (1, 1, 1, 1, 1.0)


┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_d7523fb2.jld2 saved.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:115
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_3fbcc627.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:106
┌ Warning: The Git repository ('/Users/nobuaki/Documents/Github/flexOPT') is dirty! Appending -dirty to the commit ID.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_tools.jl:71


...

┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_293ac36b.jld2 saved.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:115
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_53a50ba5.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:106
┌ Warning: The Git repository ('/Users/nobuaki/Documents/Github/flexOPT') is dirty! Appending -dirty to the commit ID.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_tools.jl:71
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_53a50ba5.jld2 saved.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:115
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_5bdd78c7.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/p

(μᶜ, μ, l_n_variable + 1, l_n_field + 1, kernels[μᶜ, μ, l_n_variable + 1, l_n_field + 1]) = (2, 2, 5, 5, 0.0)
(μᶜ, μ, l_n_variable + 1, l_n_field + 1, kernels[μᶜ, μ, l_n_variable + 1, l_n_field + 1]) = (3, 2, 5, 5, 0.0)
(μᶜ, μ, l_n_variable + 1, l_n_field + 1, kernels[μᶜ, μ, l_n_variable + 1, l_n_field + 1]) = (1, 3, 5, 5, 0.0)
(μᶜ, μ, l_n_variable + 1, l_n_field + 1, kernels[μᶜ, μ, l_n_variable + 1, l_n_field + 1]) = (2, 3, 5, 5, 0.0)
(μᶜ, μ, l_n_variable + 1, l_n_field + 1, kernels[μᶜ, μ, l_n_variable + 1, l_n_field + 1]) = (3, 3, 5, 5, 0.0)


┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_8d52e597.jld2 saved.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:115
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_a5671575.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:106
┌ Warning: The Git repository ('/Users/nobuaki/Documents/Github/flexOPT') is dirty! Appending -dirty to the commit ID.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_tools.jl:71
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_a5671575.jld2 saved.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:115
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/BsplineInt/BsplineTaylorIntegral1D_ec28e857.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/p

"GPU computation of Ajiννᶜ: done" = "GPU computation of Ajiννᶜ: done"


FieldError: FieldError: type NamedTuple has no field `nod`, available fields: `node`, `nᶜ`, `n`

In [ ]:
using Symbolics

In [ ]:
@show coefs["output"].Ajiννᶜs[1][1,1,1]
#@show size(coefs["output"].Ajiννᶜs[])

In [ ]:
@show size(coefs["output"].AjiννᶜUs)
#@show coefs["output"].ΓjkννᶜFs
@show Symbolics.expand.(coefs["output"].AjiννᶜUs[1])

# now we construct numerical operators based on the semi-symbolic rules that we constructed above

In [ ]:
models=[] # you might need to make this empty tuple first, otherwise one-member tuple can be misinterpreted
models=push!(models,seismicModelMarmousi.Vpv.*1.e3) # 1.e3 from km/s -> m/s

# in this example, the material variable is just v and we use Vpv 

# put fake Nt here for quasi-numerical operator construction


In [ ]:
models=push!(models,seismicModelMarmousi.Vpv.*1.e3) 
modelPoints = getModelPoints(models[1],pointsInTime,coefs["output"].utilities.timeMarching)#reference model dimension (this is important to explicitly define since the dimension/size of material variable array(s) can differ from the model domain)

In [ ]:
IneedExternalSources = true
maskedRegionForSourcesInSpace = nothing
# if IneedExternalSources and if the source region is localised in space then
maskedRegionForSourcesInSpace  = Array{CartesianIndex,1}(undef,0) # it is important to decalre the type of this
maskedRegionForSourcesInSpace = push!(maskedRegionForSourcesInSpace, CartesianIndex(modelPoints[1:end-1].÷2))# in Ndimension (or Ndimension  - 1 if timeMarching)
# in this example, I put a point source at the centre of the model space

forceModels =((1.0)) # if your model does not have anything special material parameters then it's how it's written

In [ ]:
# here below is still under construction

In [ ]:
function constructingNumericalDiscretisedEquations(config::Dict)
    # just a wrapper
    #@unpack semiSymbolicOpt,coordinates,modelName,models,fields,vars,famousEquationType,modelPoints,utilities, maskedRegion, NpointsUsed = config
   
    @unpack semiSymbolicCoefs, models, modelName, modelPoints, myEquationInside, maskedRegion = config
    @show config
    costfunctions,場,champsLimité=constructingNumericalDiscretisedEquations(semiSymbolicCoefs,myEquationInside,models,modelPoints,maskedRegion;initialCondition=0.0)
    numOperators=(costfunctions=costfunctions,場=場,champsLimité=champsLimité)

    #@show costfunctions
    return @strdict(numOperators)
end


In [ ]:






function constructingNumericalDiscretisedEquations(semiSymbolicCoefs,myEquationInside,models,modelPoints,maskedRegion;absorbingBoundaries=nothing,CerjanGridPoints=50,initialCondition=0.0)

    #region todo list
    #todo list
    # 
    # this function is tooooooo complicated! I think I can simplify very much this!
    #
    #
    #  need to work on the bc, same like the masked thing (limited region of source)
    #
    # absorbing boundaries : I think we can already put the bc inside the numerical operators but be careful with the time marching: search for weightingCerjan
    # 
    # need extend to 4 points with the same test functions (3 points) -> staggered grid
    #  
    # I have to include some complex initial condition for 場
    #
    # have to write:
    #  function illposedTaylorCoefficientsInversion(coordinates,multiOrdersIndices,multiPointsIndices,midPoint,Δ)
    #endregion
    
    #region general introduction
    #
    #
    # after the construction of local (semi-)symbolic expressions with linearised operators#
    # here we will read the model parameters and construct the numerical operators
    #
    # Nobuaki Fuji @ IPGP/UPC/IUF since 2024
    #
    # 
    # encouraged by Thibault Duretz @ U. Frankfurt Goethe, Kurama Okubo @ NIED
    #
    # Julia hackathon October 2024, March 2025
    #
    #

    # coordinates: Model: the real model domain; Whole: computation domain with absorbing boundaries; 
    #              Empty: Whole + some more points to avoid missing reference to the field and material (they should be just zeros)
    #

    # intermediate presentations: IPGP-CIA workshop October 2024; IPGP-ERI workshop November 2024; lighttalk @ systemI December 2024
    #             EGU @ Vienna May 2025
    #    
    #     Fuji & Duretz in preparation
    #
    #
    #
    #
    #endregion

    #region unpacking, N-dimensionalising all the models (if not timemarching, we add a dummy dimension 't' at the end)

    
    semiSymbolicsOperators=semiSymbolicCoefs["output"].AjiννᶜUs
    Ulocals=semiSymbolicCoefs["output"].Ulocals
    

    coordinates=myEquationInside.coordinates
    fields=myEquationInside.fields
    vars=myEquationInside.vars

    utilities=semiSymbolicCoefs["output"].utilities

    availablePointsConfigurations=utilities.availablePointsConfigurations
    centrePointConfigurations=utilities.centrePointConfigurations


    @unpack availablePointsConfigurations,centrePointConfigurations,localPointsIndices,localMaterials,timeMarching = utilities
    #timeMarching = utilities.timeMarching


    # not like allsame = all(s -> s == size.(models)[1], size.(models))

   
    # the last coordinate should be cosidered as time

    if !timeMarching
        localPointsIndices=CartesianIndices(Tuple([car2vec(localPointsIndices[end]);1]))
        middlepoint=CartesianIndex([car2vec(middlepoint);1]...)
        tmpT=Symbolics.variable(timeDimensionString)
        coordinates = (coordinates...,tmpT)
        #modelPoints = (modelPoints...,1)
        tmp_del = Symbolics.variable("∂"*timeDimensionString)
        tmp_del = Differential(tmpT)
        ∂ .= push!(∂,tmp_del)
        tmp_del_2 = Symbolics.variable("∂"*timeDimensionString*"²")
        tmp_del_2 = Differential(tmpT)*Differential(tmpT)
        ∂² .= push!(∂²,tmp_del_2)
    end

    #@show coordinates,∂,∂²

    localPointsSpaceIndices=CartesianIndices(Tuple(car2vec(localPointsIndices[end])[1:end-1]))
    Ndimension=length(coordinates)
    
   
    modelPoints=collect(modelPoints)

    Models=[]

    NtypeofMaterialVariables = length(vars)
    NtypeofFields = length(fields)
    NtypeofExpr = size(semiSymbolicsOperators)[end]

    Models=Array{Any,1}(undef,NtypeofMaterialVariables)
    ModelPoints=Array{Int,2}(undef,Ndimension,NtypeofMaterialVariables)

    if length(models) !== NtypeofMaterialVariables 
        @error "Each material has to have its own model"
    end
    
    
    for iVar in eachindex(vars)
        CartesianDependency=findCartesianDependency(vars[iVar],length(coordinates))
        if ndims(models[iVar]) === CartesianDependency
            @error "model parameter dimension is not what you declared in the equation!"
        end
        if sum(CartesianDependency) === 0 # when it is a constant
            tmpModel=Array{Any,Ndimension}(undef,(ones(Int, Ndimension)...)...)
            ModelPoints[:,iVar] = ones(Int, Ndimension)
            tmpModel[vec2car(ones(Int, Ndimension))] = models[iVar]
            Models[iVar]=tmpModel
        else
            #@show models[iVar],iVar,CartesianDependency, vars[iVar]
            newCoords=expandVectors(size(models[iVar]),CartesianDependency)
            ModelPoints[:,iVar] = newCoords
 
            tmpModel=reshape(models[iVar],newCoords...)
            Models[iVar]=tmpModel

            for iCoord in eachindex(newCoords)
                if newCoords[iCoord]!== modelPoints[iCoord] && newCoords[iCoord] !== 1
                    @error "the model should have the same dimension! (or constant)"
                end
            end
        end
     
    end

    #endregion

    #region construction of the fields, wholeRegionPoints with absorbingBoundaries (need to check if this works)

    wholeRegionPoints = nothing

    if absorbingBoundaries === nothing
        wholeRegionPoints=modelPoints
        absorbingBoundaries = zeros(Int,2, Ndimension)
    elseif absorbingBoundaries === "CerjanBoundary"
        wholeRegionPoints=modelPoints
        absorbingBoundaries = ones(Int,2, Ndimension-1)*CerjanGridPoints
        absorbingBoundaries=[absorbingBoundaries; 0 0]
        wholeRegionPoints=modelPoints.+sum(absorbingBoundaries,1) 
    else
        # absorbingBoundaries should be two column array 
        if size(absorbingBoundaries)[1] !== 2
            @error "you have to give us the left and right values for absorbing boundaries"
        elseif size(absorbingBoundaries)[2] !== size(modelPoints)[1] && !timeMarching
            @error "you have to give us the values for each direction for absorbing boundaries"
        elseif size(absorbingBoundaries)[2] === size(modelPoints)[1]-1 && timeMarching
            absorbingBoundaries=[absorbingBoundaries; 0 0]
        end
        wholeRegionPoints=modelPoints.+sum(absorbingBoundaries,1) 
    end


    # some useful stuff


    spacePointsUsed=car2vec(localPointsIndices[end])[1:end-1]
    timePointsUsedForOneStep=car2vec(localPointsIndices[end])[end]
    wholeRegionPointsSpace=wholeRegionPoints[1:end-1]
   
 
    # we need to put the left and right regions in order that centre ν configuration can pass

    #emptyRegionPointsSpace=wholeRegionPointsSpace.+ 2 .* spacePointsUsed

    #場dummy=Array{Any,2}(undef,NtypeofFields,timePointsUsedForOneStep)
    場 = Array{Any,2}(undef,NtypeofFields,timePointsUsedForOneStep)
    

    for it in 1:timePointsUsedForOneStep
        for iField in eachindex(fields)
            newstring=split(string(fields[iField]),"(")[1]*"_mod"*"_t="*string(it)
            場[iField,it]=Symbolics.variables(Symbol(newstring),Base.OneTo.(Tuple(wholeRegionPointsSpace))...)
        end
    end

    #since everything is super clumsy, here we make several useful functions to change one coordinate to another
    
    conv=spaceCoordinatesConversionfunctions(absorbingBoundaries[:,1:end-1], Ndimension-1)

    #endregion 

    #region making a maskingField (for limited source areas, boundary conditions, etc.)

    maskingField=Array{Any,Ndimension-1}(undef,Tuple(wholeRegionPointsSpace)) # maskingField is defined only for whole domain
    champsLimité = nothing
    if maskedRegionInSpace === nothing
        maskingField .= 1.0
    elseif typeof(maskedRegionInSpace) === Array{CartesianIndex,1}
        champsLimité = Array{Any,2}(undef,NtypeofFields,timePointsUsedForOneStep)
        for it in 1:timePointsUsedForOneStep
            for iField in eachindex(fields)
                newstring=split(string(fields[iField]),"(")[1]*"_mod_limited"*"_t="*string(it)
                champsLimité[iField,it] = Array{Any,1}(undef,length(maskedRegionInSpace))
            end
        end
        maskingField .= 0.0
        tmpIndex=1
        for iSpace in maskedRegionInSpace
            jSpace = conv.model2whole(iSpace)
            maskingField[jSpace] =1.0
            for it in 1:timePointsUsedForOneStep
                for iField in eachindex(fields)
                    
                    #tmpChampsLimitéContents= (jSpace,場[iField,it][jSpace])
                    champsLimité[iField,it][tmpIndex]=場[iField,it][jSpace]
                end
            end
            tmpIndex += 1
        end
    else
        @error "maskedRegionInSpace should be a 1D array of CartesianIndex (if it is CartesianIndices, you need to collect(Tuple()))"
    end


    #endregion

    #region relative ν to be considered, especially around the boundaries, useful for the following sections

    PointsSpace=CartesianIndices(Tuple(wholeRegionPointsSpace))
    NpointsSpace=length(PointsSpace) # number of points in space

    NtestfunctionsInSpace=NpointsSpace # this assumption is valid only for test functions related to grid points

    νWhole=Array{Any,1}(undef,NtestfunctionsInSpace) # the coordinate in wholeRegionPointsSpace: for the moment mapping from testfunction to ν is bijective

    # below is only for the bijective projection between test functions and ν

    for iPoint in eachindex(νWhole)

        νWhole[iPoint] = PointsSpace[iPoint] # this should be not true for higher B-spline test functions

    end


    νRelative=Array{Any,1}(undef,NtestfunctionsInSpace) # the relative coordinate to take (the coordinate used for the semi-symbolic operator derivation)
    νRelative.=middlepoint


    
    #endregion

    #region we construct the numerical operators for each test function that is related to its corresponding point

    # first we compute the νRelative more seriously if testOnlyCentre we might do nothing at all

    if testOnlyCentre
        # If we compute only the operators without boundaries, we use kind of 'truncated' crazy operators 
        # derived at the centre point and we do not talk about it, just believe the absorbing boundaries
        # like, tant pis, il n'y a pas de points donc j'ignore juste !

        # maybe we do not do anything!

    else

        # we use this clause only if we are interested in a serious boundary conditions, 
        # i.e. not the artificial cartesian box boundaries (we can just apply some stupid absorbing boundaries in that case)
        # Hence this clause should be more generalised even it kills the performance
        #  like, we give an array of free surface or discontinuities in CartesianIndex arrays 
        # and we look all the points concerned

        # we need to explore everywhere in the wholeRegionPoints! Free surface etc. should be very much affected 
        #
      
        boundaryPointsSpace=[]
        for iDimSpace in 1:Ndimension-1 # we take care of the boundaries of the Cartesian box (it should be the same for internal/external topography)
            # points concerned
            leftstart=1
            leftend=spacePointsUsed[iDimSpace]÷2
            rightstart=NpointsSpace[iDimSpace]-spacePointsUsed[iDimSpace]÷2+1
            rightend=NpointsSpace[iDimSpace]
            # suppose that the domain is sufficiently big (maybe it is not the case for some crazy topography ...)
            for iCoord in range(leftstart,leftend)
                


            end

            for iCoord in range(rightstart,rightend)

            end
        end
    end

    # here the number of test functions should not be necessarily the number of points but I will work later

    costFunctions=Array{Any,2}(undef,NtypeofExpr,NtestfunctionsInSpace)
    @show size(costFunctions)
    #@show semiSymbolicsOperators
    #@show localMaterials[1,15],localFields,size(localFields)
    #@show Models[1][10,15,1]

    for iTestFunctions in eachindex(νWhole)
        # here each test function is connected to one ν point 
        # We need to be careful that this can be no more true for different basis functions other than linear B-spline
        iPoint = iTestFunctions 
        νtmpWhole=νWhole[iPoint]
        # be careful with the two lines above, they are based on the assumption that each test function is linked to one collocated point


        #νtmpModel=conv.whole2model(νtmpWhole)
        νᶜtmpWhole = localPointsSpaceIndices .+ (νtmpWhole - carDropDim(νRelative[iPoint])) # this is the shift vector
        νᶜtmpModel = conv.whole2model.(νᶜtmpWhole)

        # examine νᶜtmpWhole if it is out of the range 



        for iExpr in eachindex(semiSymbolicsOperators[1,:])

            tmpMapping=Dict()


            for iT in 1:timePointsUsedForOneStep
                
                for iVar in eachindex(vars)
                    
                    spaceModelBouncedPoints=ModelPoints[1:end-1,iVar]

                    if ModelPoints[end,iVar] > 1
                        iiT=iT
                    else
                        iiT = 1
                    end


                    # model parameters should be bounced at the whole region limits
                    νᶜtmpModelTruncated = BouncingCoordinates.(νᶜtmpModel, Ref(spaceModelBouncedPoints))

                    for jPoint in νᶜtmpWhole
                        jPointLocal = jPoint - νtmpWhole + carDropDim(νRelative[iPoint])
                        jPointTLocal = carAddDim(jPointLocal,iT)
                        linearjPointTLocal=LinearIndices(localPointsIndices)[jPointTLocal]
              
                        tmpMapping[localMaterials[iVar,linearjPointTLocal]] = Models[iVar][carAddDim(νᶜtmpModelTruncated[jPointLocal],iiT)]
                        
                    end

                end

                for jPoint in νᶜtmpWhole
                    #@show iPoint, jPoint
                    jPointLocal = jPoint - νtmpWhole + carDropDim(νRelative[iPoint])
                    jPointTLocal = carAddDim(jPointLocal,iT)
                    linearjPointTLocal=LinearIndices(localPointsIndices)[jPointTLocal]
                    #jPointT=carAddDim(jPoint,iT)
                    #linearjPointT=LinearIndices(localPointsIndices)[jPointT]
                    for iField in eachindex(fields)
                        if is_all_less_than_or_equal(CartesianIndex(ones(Int, Ndimension-1)...),conv.whole2model(jPoint)) && is_all_less_than_or_equal(conv.whole2model(jPoint),vec2car(ModelPoints[1:end-1]))
                            # when it is inside the model domain box
                            tmpMapping[localFields[linearjPointTLocal,iField]] = 場[iField,iT][jPoint]*maskingField[jPoint]

                        elseif is_all_less_than_or_equal(νWhole[1],jPoint) && is_all_less_than_or_equal(jPoint,νWhole[end])
                            # if it is in the absorbing boundary zones we apply a simple Cerjan
                            if iT === timePointsUsedForOneStep # the last one (the future) will be using un-weighted operators
                                tmpMapping[localFields[linearjPointTLocal,iField]] = 場[iField,iT][jPoint]*maskingField[jPoint]
                            else
                                distance2 = distance2_point_to_box(conv.whole2model(jPoint),CartesianIndex(ones(Int, Ndimension-1)...), vec2car(ModelPoints[1:end-1]))
                                tmpMapping[localFields[linearjPointTLocal,iField]] = 場[iField,iT][jPoint]*maskingField[jPoint]*CerjanBoundaryCondition(distance2)
                            end
                        else
                            tmpMapping[localFields[linearjPointTLocal,iField]]=0.
                            #jPoint, νWhole[1],νWhole[end]
                        end
                    end
                end

                        

                        #場[iField,it]=string_as_varname(newstring, Array{Any,Ndimension-1}(undef,Tuple(wholeRegionPointsSpace)))
                        #
                        #νᶜtmpWholeMissing = ReplacerHorsLimiteParMissing(νᶜtmpWhole,PointsSpace[end])
                        # field values are defined only at the whole region and not at the Empty
                        #replace!(x -> x>0.2 ? missing : x, Array{Union{Float64, Missing}}(A) )
                        #replace!(x -> )
                        
                        
                
               
            end
            tmpAddress = nothing
            if testOnlyCentre
                tmpAddress = 1
            else
                tmpAddress=carDropDim(νRelative[iPoint])
            end
            costFunctions[iExpr,iTestFunctions]=substitute(semiSymbolicsOperators[tmpAddress,iExpr],tmpMapping)
            # be careful that semiSymbolicsOperators could be 2D
        end
    end

    #@show costFunctions

    #endregion

    return costFunctions,場,champsLimité

end



In [ ]:
lhsConfigurations = @strdict semiSymbolicCoefs=coefs models modelName="MarmousiVp" modelPoints myEquationInside maskedRegion=nothing
#numOperators = myProduceOrLoad(constructingNumericalDiscretisedEquations,lhsConfigurations,"numOperators","lhs")